<a href="https://colab.research.google.com/github/HaleBera/Container-Attack-Dataset/blob/main/Sprint_1_1_VulfocusAutomate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Sprint 1*** =>  "*RAISE OF VULFOCUS*" 

  1. It can only start a single image on vulfocus, and carry out a single attack (Sprint 1.1 => make it to run multicontainers, so that multiple nuclei attacks) **(COMPLETED)** ***(by Buğra Alparslan  https://github.com/iambugra/vulfocus_automater
)***
  2. Not capable of taking input-output from the command line (Sprint 1.2 => convert some of the staticly assigned variables to be assigned by the parameters that is given together with the run command)
  3. Currently, it requests to the web (vulfocus.io), we need to make it run locally (Sprint 1.3 => https://github.com/fofapro/vulfocus/blob/master/INSTALL.md 'den bul bir yolunu :) )





***Sprint 1.1:*** INPUT & OUTPUT  **(COMPLETED  https://github.com/iambugra/vulfocus_automater
)**

Some variables should be given as an input to the program:

1.   Sytem Path Info (String): '/home/hale/Desktop/TEST_ENVIRONMENT/Vulfocus/vulfocus_automater-master/vulfocusAutomate'
2.   File of Image List's Path Info (String): './readyToAttackImages.txt'
3.   File of Nuclei Attack List's Path Info (String): './readyToAttackNucleiTemplates.txt'
4.   File where Nuclei exports its results (String):'./nuclei_results'
     (We need a mini coding that will add the date-time info at the end of the results file name i.e. nuclei_results_10-04-2022_14:26)

     

 

***Sprint 1.2:*** MULTI-CONTAINER

Some coding segments are needed to be rewrited to be able to start multiple containers at the same time (Or if is not possible "at the same time", atleast, we may try to *properly* start and close one-by-one)


*   ## START CONTAINER


> ```
> # startedContainer = clinet.start_container(imageLine)
> ```
  
      
*   ## STOP AND DELETE CONTAINER
        #print(clinet.stop_container(imageLine))
        #print(clinet.delete_container(imageLine))

        

NOTE: Currently it enables max 3 containers active at the same time

***Sprint 1.3:*** LOCALIZE VULFOCUS

https://github.com/fofapro/vulfocus/blob/master/INSTALL.md

  - Install a Mysql server local
  - Integrate it with Vulfocus
  - May be try it installing on a VM





In [ ]:
import parse_args
import sys, os
from src.vulfocus.vulfocusClient import VulfocusClient
import credentials
from datetime import datetime

arguments = parse_args.parse()

imageFileToRead = open(arguments['image-file'], 'r')
nucleiTemplateFileToRead = open(arguments['template-file'], 'r')
output_dir = arguments['output-dir']
VERBOSE = arguments['verbose']


sys.path.insert(0, os.path.dirname(os.path.realpath(__file__)))
print( "Current directory: "+ os.path.dirname(os.path.realpath(__file__)))


clinet = VulfocusClient(username=credentials.USERNAME,licence=credentials.LICENCE)
images = clinet.get_images()


if(VERBOSE):
    print(images)

count = 0
while True:
    count += 1
 
    # Get next line from both files
    imageLine = imageFileToRead.readline()
    nucleiLine =  nucleiTemplateFileToRead.readline()
 
    # if line is empty
    # end of file is reached
    if not imageLine:
        break
    print("Starting image name: {} nuclei attack: {}".format(imageLine.strip(), nucleiLine.strip()))

    ## START CONTAINER
    startedContainer = clinet.start_container(imageLine)
    if(VERBOSE):
        print(startedContainer)
    startedHost = startedContainer.host
    startedHostWithoutPort = startedHost.split(':')[0]
    startedPorts = startedContainer.port

    # we dont know which port is working for nuclei attack
    # why not try all :D
    for port in startedPorts.values():

        current_time = datetime.now()
        
        ## NUCLEI ATTACK STARTS HERE
        # nucleiCommand = "nuclei -u http://{} -t {} -debug -markdown-export ./nuclei_results".format(startedHostWithoutPort +":"+ port, nucleiLine.strip())
        nucleiCommand = f'nuclei -u http://{startedHostWithoutPort}:{port} -t {nucleiLine.strip()} -debug -markdown-export {output_dir}_{current_time.year}-{current_time.month}-{current_time.day}_{current_time.hour}:{current_time.minute}'
        cwd = os.path.dirname(os.path.realpath(__file__))

        print("Switching to current working directory")
        commandResult1 = os.system("cd "+cwd)
        
        print("Running nuclei attack with port: {} :D Hope it will find something.".format(port))
        try:
            commandResult2 = os.system(nucleiCommand)
        except:
            print("Failed to execute nuclei command")
        ## NUCLEI ATTACK ENDS HERE

        ## STOP AND DELETE CONTAINER
        #print(clinet.stop_container(imageLine))
        #print(clinet.delete_container(imageLine))





ModuleNotFoundError: ignored